In [ ]:
!pip install keras-tuner --upgrade

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import skimage, os
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.misc
import numpy as np
from glob import glob
from skimage.io import imread
import re
from random import shuffle
import random

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as anim

import imageio
from skimage.transform import resize

import copy
from scipy import ndimage as nd
import nibabel as nib
import itertools
import cv2

import tensorflow as tf
from tensorflow.keras.layers import Input, concatenate, Conv3D, MaxPooling3D, Conv3DTranspose, AveragePooling3D, ZeroPadding3D
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.metrics import AUC

import math

In [ ]:
all_images = glob(os.path.join('/content/drive/MyDrive/Data/guangdi_1/Hospital A','*.nii.gz'))
df = pd.read_excel('/content/drive/MyDrive/Data/guangdi_1/clinic.xlsx', sheet_name='Hospital A')

all_images_b = glob(os.path.join('/content/drive/MyDrive/Data/guangdi_1/Hospital B','*.nii.gz'))
df_val = pd.read_excel('/content/drive/MyDrive/Data/guangdi_1/clinic.xlsx', sheet_name='Hospital B')

len(all_images), df.shape[0], len(all_images_b), df_val.shape[0]

In [ ]:
def match_img(row, t='train'):

  img_path = np.nan
  if t == 'train':
    name = '_'.join(row['Name'].strip().upper().split(' '))
    cur_all_images = all_images
  else:
    name = '_'.join(row['缩写'].strip().upper().split(' '))
    cur_all_images = all_images_b

  for img in cur_all_images:
    org_img = img
    img = img.split('/')[-1]
    f_i = re.search(r"\d", img).start()
    cur_name = img[: f_i-1]
    if name == cur_name:
      img_path = org_img
      break

  return img_path

df['img_path'] = df.apply(lambda x: match_img(x, 'train'), axis=1)
df_val['img_path'] = df_val.apply(lambda x: match_img(x, 'test'), axis=1)

In [ ]:
show_ids = np.random.randint(df.shape[0])
print(df.iloc[show_ids]['Name'])
print(df.iloc[show_ids]['IDx'])
print(df.iloc[show_ids]['img_path'])

print('------------------------------')
show_ids = np.random.randint(df_val.shape[0])
print(df_val.iloc[show_ids]['缩写'])
print(df_val.iloc[show_ids]['img_path'])

In [ ]:
df = df.dropna().reset_index(drop=True)
df['label'] = df['预后'].apply(lambda x: int(1) if x == 2 else int(0))
df = df[['Name', 'img_path', 'label']]

df_val = df_val.fillna(method='ffill')
df_val['label'] = df_val['预后'].apply(lambda x: int(1) if x == 2 else int(0))
df_val = df_val[['缩写','img_path', 'label']]

df.shape, df_val.shape

In [ ]:
img_rows = 128
img_cols = 128
img_depth = 64


def resize_volume(image_p):
    img = copy.deepcopy(image_p)
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = img_depth
    desired_width = img_rows
    desired_height = img_cols
    z_ids = np.linspace(0, img.shape[2]-1, desired_depth)
    z_ids = list(map(lambda x: int(x), z_ids))
    img3d = np.stack([cv2.resize(img[:, :, i], (desired_width, desired_height)) for i in z_ids]).T
    if np.min(img3d) < np.max(img3d):
      img3d = img3d - np.min(img3d)
      img3d = img3d / np.max(img3d)
    if img3d.shape[-1] < desired_depth:
      n_zero = np.zeros((desired_height, desired_width, desired_depth - img3d.shape[-1]))
      img3d = np.concatenate((img3d,  n_zero), axis = -1)
    return img3d


def load_dicom_images_3d(img_path):

    img3d = nib.load(img_path).get_fdata()
    img3d = resize_volume(img3d)

    return np.expand_dims(img3d,-1)

In [ ]:
from tensorflow.keras.utils import Sequence

class Dataset(Sequence):
    def __init__(self,df,is_train=True,batch_size=2,shuffle=True):
        self.idx = df.index
        self.paths = df["img_path"].values
        self.y = df['label'].values
        self.is_train = is_train
        self.batch_size = batch_size
        self.shuffle = shuffle
    def __len__(self):
        return math.ceil(len(self.idx)/self.batch_size)
   
    def __getitem__(self,ids):
        id_path= self.paths[ids]
        batch_paths = self.paths[ids * self.batch_size:(ids + 1) * self.batch_size]
        
        if self.y is not None:
            batch_y = self.y[ids * self.batch_size: (ids + 1) * self.batch_size]
        
        if self.is_train:
            list_x =  [load_dicom_images_3d(x) for x in batch_paths]
            batch_X = np.stack(list_x, axis=0)
            return batch_X, batch_y
        else:
            list_x = load_dicom_images_3d(id_path)
            batch_X = np.stack(list_x, axis=0)
            return batch_X
    
    def on_epoch_end(self):
        if self.shuffle and self.is_train:
            ids_y = list(zip(self.idx, self.y))
            shuffle(ids_y)
            self.idx, self.y = list(zip(*ids_y))

In [ ]:
from tqdm.notebook import tqdm

def get_all_data_for_train():
  x_train = []
  y_train = []
  labels = pd.get_dummies(df['label']).values
  for ids, row in tqdm(df.iterrows()):
    paths = row['img_path']
    x = load_dicom_images_3d(paths)
    if x.shape != (img_rows, img_cols, img_depth, 1):
      continue
    x_train.append(x)
    y_train.append(labels[ids])
  
  return np.array(x_train), np.array(y_train)

x_train, y_train = get_all_data_for_train()
x_train.shape, y_train.shape

In [ ]:
def check_image_mask(row):
  img_path = row['img_path']

  img = load_dicom_images_3d(img_path)

  return 1 if img.shape == (img_rows, img_cols, img_depth, 1) else 0

df['check'] = df.apply(check_image_mask, axis=1)
df_val['check'] = df_val.apply(check_image_mask, axis=1)

In [ ]:
df = df[df['check'] == 1].reset_index(drop=True)
df.shape

In [ ]:
train_dataset = Dataset(df)
val_dataset = Dataset(df_val)

In [ ]:
show_ids = np.random.randint(100)
images, labels = train_dataset[show_ids]
print("Dimension of the img is:", images.shape)

In [ ]:
import keras_tuner as kt

def make_model(hp):
    inputs = keras.Input((img_rows, img_cols, img_depth, 1))

    x = keras.layers.Conv3D(filters=hp.Int('units_Conv_1_' + str(0),
                                            min_value=16,
                                            max_value=64,
                                            step=4),
                            kernel_size=3,
                            activation="relu", padding='same',
                            name="Conv_1")(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.layers.MaxPool3D(pool_size=2, padding='same')(x)

    conv_1 = x

    x = keras.layers.Conv3D(filters=hp.Int('units_conv2_' + str(1),
                                            min_value=16,
                                            max_value=128,
                                            step=4),
                            kernel_size=3,
                            activation="relu", padding='same',
                            name="Conv_2")(x)
    x = layers.BatchNormalization()(x)
    x = keras.layers.MaxPool3D(pool_size=2, padding='same')(x)


    con3_f = hp.Int('units_conv3_' + str(2), min_value=32, max_value=256, step=4)

    x = keras.layers.Conv3D(filters=con3_f,
                            kernel_size=3,
                            activation="relu", padding='same',
                            name="Conv_3")(x)
    x1 = keras.layers.Conv3D(filters=con3_f, kernel_size=1, strides=2, activation="relu", padding='same', name="Conv_input")(conv_1)
    x = keras.layers.concatenate([x, x1])
    x = layers.BatchNormalization()(x)

    x = keras.layers.MaxPool3D(pool_size=1, strides=4, padding='same')(x)
    
    x = layers.Dropout(
        hp.Float('dense_dropout', min_value=0., max_value=0.7)
    )(x)
    x = keras.layers.Flatten()(x)
    x = layers.Dense(
        units=hp.Int('num_dense_units', min_value=8, max_value=64, step=8),
        activation='relu'
    )(x)

    outputs = keras.layers.Dense(2, activation="softmax")(x)

    model = keras.Model(inputs, outputs)

    cat_acc = tf.keras.metrics.BinaryAccuracy(name='acc')

    model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=[cat_acc]
    )
    model.summary()
    return model

In [ ]:
import keras_tuner

tuner = kt.tuners.BayesianOptimization(
    make_model,
    objective=keras_tuner.Objective("val_acc", direction="max"),
    max_trials=10, 
    overwrite=True)

callbacks=[keras.callbacks.EarlyStopping(monitor='val_acc', mode='max', patience=5, baseline=0.9)]

tuner.search(x_train, y_train, validation_split=0.1 ,callbacks=callbacks, verbose=1, epochs=50, batch_size=2)

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
best_model = make_model(best_hp)
keras.utils.plot_model(best_model, show_shapes=True)

In [ ]:
def get_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=img_rows, height=img_cols, depth=img_depth)

In [ ]:
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
# tf.keras.metrics.AUC(name='auc')
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=[tf.keras.metrics.AUC(name='auc')],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint("3d_image_classification.h5", save_best_only=True, monitor='val_auc', mode='max', verbose=1)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_auc", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 100
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

In [ ]:
from tqdm.notebook import tqdm

test_dataset = Dataset(df_val, is_train=False, batch_size=1)

preds = []
for i in tqdm(range(len(test_dataset))):
  preds.append(model.predict(np.expand_dims(test_dataset[i], axis=0))[0][0])

In [ ]:
avg_pred = np.mean(preds)
avg_pred

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

y_test = df_val['label']
preds_int = []

for v in preds:
  if v > avg_pred:
    preds_int.append(1)
  else:
    preds_int.append(0)

print(classification_report(y_test, preds_int))
print(f'AUC is {roc_auc_score(y_test, preds)}')